https://huggingface.co/Qwen/Qwen2-Audio-7B-Instruct

In [1]:
from io import BytesIO
from urllib.request import urlopen
import librosa
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor

model_path = "/home/xwchen/r1-aqa/exp/model/checkpoint-200"


In [2]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "/home/xwchen/data/mmau/test-mini-audios/6aee68bf-6629-442b-981d-ae8195597c8e.wav"},
        {"type": "text", "text": """Based on the given audio, identify the source of the speech. Please choose an answer from the following options: [ "Radio", "Fire truck", "Construction site", "Airplane"]. The answer is not "Construction site". Let's think step by step. Output the thinking process in <think> </think> and final answer in <answer> </answer>."""},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                audios.append(librosa.load(
                    ele['audio_url'], 
                    sr=processor.feature_extractor.sampling_rate)[0]
                )

inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
inputs.input_ids = inputs.input_ids.to("cuda")

generate_ids = model.generate(**inputs, max_length=1024)
generate_ids = generate_ids[:, inputs.input_ids.size(1):]

response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
response

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


/home/xwchen/miniconda3/envs/verl/lib/python3.11/site-packages/transformers/generation/utils.py:2105: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'<think> The sound seems to be coming from a distance, possibly from a building or a vehicle that is moving away. It could be from a radio or a loudspeaker broadcasting something. Another possibility is that it is from an emergency vehicle like a fire truck or an airplane that is in the distance. </think> \n\n<answer> Radio. </answer>'

In [ ]:
text